In [1]:
from scapy.all import * 
import pandas as pd 
import numpy as np 
import binascii 
import seaborn as sns
sns.set(color_codes=True)

ModuleNotFoundError: No module named 'scapy'

In [ ]:
num_of_packets_to_sniff = 100
pcap = sniff(count=num_of_packets_to_sniff)
print(type(pcap))
print(len(pcap))
print(pcap)
pcap[0]

In [ ]:
cd Desktop


In [ ]:
pcap = pcap + rdpcap("/home/jishnusaurav/Desktop/phishingattack.pcap")

In [ ]:
pcap


In [ ]:
ethernet_frame = pcap[101]
ip_packet = ethernet_frame.payload
segment = ip_packet.payload
data = segment.payload 
print(ethernet_frame.summary())
print(ip_packet.summary())
print(segment.summary())
print(data.summary()) 
ethernet_frame.show()

In [ ]:
ethernet_type = type(ethernet_frame)
ip_type = type(ip_packet)
tcp_type = type(segment)
print("Ethernet",pcap[ethernet_type])
print("IP", pcap[ip_type])
print("TCP", pcap[tcp_type])


from scapy.layers.l2 import Ether
from scapy.layers.inet import IP
from scapy.layers.inet import TCP, UDP

print("UDP", pcap[UDP])

In [ ]:
ip_fields = [field.name for field in IP().fields_desc]
tcp_fields = [field.name for field in TCP().fields_desc]
udp_fields = [field.name for field in UDP().fields_desc]

dataframe_fields = ip_fields + ['time'] + tcp_fields + ['payload','payload_raw','payload_hex']

df = pd.DataFrame(columns=dataframe_fields)
for packet in pcap[IP]:
    field_values = []
    for field in ip_fields:
        if field == 'options':
            field_values.append(len(packet[IP].fields[field]))
        else:
            field_values.append(packet[IP].fields[field])
    
    field_values.append(packet.time)
    
    layer_type = type(packet[IP].payload)
    for field in tcp_fields:
        try:
            if field == 'options':
                field_values.append(len(packet[layer_type].fields[field]))
            else:
                field_values.append(packet[layer_type].fields[field])
        except:
            field_values.append(None)
    
    field_values.append(len(packet[layer_type].payload))
    field_values.append(packet[layer_type].payload.original)
    field_values.append(binascii.hexlify(packet[layer_type].payload.original))
  
    df_append = pd.DataFrame([field_values], columns=dataframe_fields)
    df = pd.concat([df, df_append], axis=0)

df = df.reset_index()
df = df.drop(columns="index")

In [ ]:
print(df.iloc[0])

print(df.shape)

df.head()

df.tail()
df['src']

df[['src','dst','sport','dport']]

In [ ]:
print("# Top Source Address")
print(df['src'].describe(),'\n\n')

print("# Top Destination Address")
print(df['dst'].describe(),"\n\n")

frequent_address = df['src'].describe()['top']

print("# Who is Top Address Speaking to?")
print(df[df['src'] == frequent_address]['dst'].unique(),"\n\n")

print("# Who is the top address speaking to (Destination Ports)")
print(df[df['src'] == frequent_address]['dport'].unique(),"\n\n")

print("# Who is the top address speaking to (Source Ports)")
print(df[df['src'] == frequent_address]['sport'].unique(),"\n\n")

In [ ]:
print("Unique Source Addresses")
print(df['src'].unique())

print()

print("Unique Destination Addresses")
print(df['dst'].unique())

In [ ]:
print("Unique Source Addresses")
print(df['src'].unique())

print()

print("Unique Destination Addresses")
print(df['dst'].unique())

In [ ]:
source_addresses = df.groupby("src")['payload'].sum()

source_addresses.plot(kind='barh',title="Addresses Sending Payloads",figsize=(15,15))

In [ ]:
destination_payloads = df.groupby("dport")['payload'].sum()
destination_payloads.plot(kind='barh',title="Destination Ports (Bytes Received)",figsize=(15,15))

In [ ]:

frequent_address_df = df[df['src'] == frequent_address]
x = frequent_address_df['payload'].tolist()
sns.barplot(x="time", y="payload", data=frequent_address_df[['payload','time']],
            label="Total", color="b").set_title("History of bytes sent by most frequent address")

In [ ]:
frequent_address_df = df[df['src']==frequent_address]

frequent_address_groupby = frequent_address_df[['src','dst','payload']].groupby("dst")['payload'].sum()

frequent_address_groupby.plot(kind='barh',title="Most Frequent Address is Speaking To (Bytes)",figsize=(8,5))

suspicious_ip = frequent_address_groupby.sort_values(ascending=False).index[0]
print(suspicious_ip, "May be a suspicious address")

suspicious_df = frequent_address_df[frequent_address_df['dst']==suspicious_ip]